# Use the class


In [2]:
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
import jax
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error as mse
from scipy.optimize import minimize
from IPython.display import clear_output
import random 
from pennylane.optimize import AdamOptimizer,QNSPSAOptimizer
from utils import *
import os

from autoencoder3 import Autoencoder_composite

In [4]:
w=np.load('runs/50/weights0.npy')

In [7]:
w

tensor([2.89081478, 0.77407855, 1.70496178, 2.58332324, 0.96720803,
        0.42745292, 0.41065612, 0.08929387, 2.80106664, 0.64908105,
        1.00190616, 1.72176301, 0.01607606, 2.9855969 , 1.97631669,
        2.95476604, 1.93619514, 0.94944239, 0.07750568, 3.08060336,
        2.18078208, 1.23118806, 1.07954836, 0.09719232, 1.96867049,
        1.83231258, 2.8391974 , 2.2038312 , 1.24507511, 2.02162933,
        1.22385621, 1.47010934, 0.29521394, 1.78422153, 2.14788198,
        3.10658312], dtype=float32, requires_grad=True)

In [5]:
for wei in w:
    plt.plot(range(len(wei)),wei)

TypeError: len() of unsized object